### Connecting to PostgreSQL

In [16]:
##!pip install psycopg2
import psycopg2
import pandas as pd

In [17]:
##conn = psycopg2.connect("dbname=postgres user=postgres password=2416")

params = {
    "host" : "localhost",
    "port" : 5432,
    "database" : "postgres",
    "user": "postgres",
    "password" : "2416"
}

def connect2postgres(params):
    conn = None
    try:
        print("Connecting to postgresql...")
        conn = psycopg2.connect(**params)
    except (Exception, psycop2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("opened database succesfully !")
    return conn    

conn = connect2postgres(params)

Connecting to postgresql...
opened database succesfully !


In [18]:
select_query = '''select 
                                "id", st_x(st_centroid("geom")) x, st_y(st_centroid("geom")) y, "STATE_NAME", "STATE_FIPS", "SUB_REGION", "STATE_ABBR", "LAND_KM", "WATER_KM", "PERSONS", "FAMILIES", 
                                "HOUSHOLD", "MALE", "FEMALE", "WORKERS", "DRVALONE", "CARPOOL", "PUBTRANS", "EMPLOYED", "UNEMPLOY", "SERVICE", 
                                "MANUAL", "P_MALE", "P_FEMALE", "SAMP_POP"  from "USAPopulation" '''

column_names = ["id", "x", "y" , "STATE_NAME", "STATE_FIPS", "SUB_REGION", "STATE_ABBR", "LAND_KM", "WATER_KM", "PERSONS", "FAMILIES", 
                "HOUSHOLD", "MALE", "FEMALE", "WORKERS", "DRVALONE", "CARPOOL", "PUBTRANS", "EMPLOYED", "UNEMPLOY", "SERVICE", 
                "MANUAL", "P_MALE", "P_FEMALE", "SAMP_POP"]

def postgres2dataframe(conn, select_query, column_names):
    
    cursor = conn.cursor()
    
    try:
        cursor.execute(select_query)
    
    except (Exception, psycop2.DatabaseError) as error:
        print("Error: %s" %error)
        cursor.close()
        return 1
    
    values = cursor.fetchall()
    cursor.close()
    
    df = pd.DataFrame(values, columns=column_names)
    return df

df = postgres2dataframe(conn, select_query, column_names)
 
df.head()



id           x          y STATE_NAME STATE_FIPS SUB_REGION STATE_ABBR  \
0   1  -89.203686  40.063972   Illinois         17    E N Cen         IL   
1   9  -86.826357  32.794685    Alabama         01    E S Cen         AL   
2   2  -92.480020  38.363948   Missouri         29    W N Cen         MO   
3   3 -111.663882  34.291646    Arizona         04        Mtn         AZ   
4   4  -97.509148  35.583909   Oklahoma         40    W S Cen         OK   

      LAND_KM  WATER_KM     PERSONS  ...   DRVALONE   CARPOOL  PUBTRANS  \
0  143986.610  1993.335  11430602.0  ...  3741715.0  652603.0  538071.0   
1  131443.119  4332.268   4040587.0  ...  1374395.0  267191.0   13279.0   
2  178445.951  2100.115   5117073.0  ...  1816079.0  312042.0   47129.0   
3  294333.462   942.772   3665228.0  ...  1178320.0  239083.0   32856.0   
4  177877.536  3170.998   3145585.0  ...  1078894.0  190902.0    7817.0   

    EMPLOYED  UNEMPLOY    SERVICE    MANUAL  P_MALE  P_FEMALE   SAMP_POP  
0  5417967.0  385040.0  1360159.0  828906.0   0.486     0.514  1747776.0  
1  1741794.0  128587.0   474797.0  360499.0   0.479     0.521   630032.0  
2  2367395.0  155388.0   659782.0  386746.0   0.482     0.518   864999.0  
3  1603896.0  123902.0   455896.0  185109.0   0.494     0.506   468178.0  
4  1369138.0  100931.0   405802.0  205601.0   0.487     0.513   528722.0  

[5 rows x 25 columns]

In [19]:
conn.close()